In [1]:
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
import nibabel as nib
import os
import pandas as pd
import scipy.linalg as la
%matplotlib inline

In [2]:
# Process is
# 1. Load runs
# 2. Clean up signal
# 3. zscore runs
# 4. stack
# 5. compute isc

In [3]:
def load_gifti(fn):
    data = nib.load(fn)
    data = np.vstack([d.data for d in data.darrays])
    return data

In [4]:
data_dir = os.path.abspath('../outputs/fmriprep')


def get_subjects():
    fns = sorted(glob(os.path.join(data_dir, 'sub-*/')))
    fns = [fn.split('/')[-2] for fn in fns]
    return fns


def load_data(subject):
    print(f"Loading data for subject {subject}")
    datadir = f"{data_dir}/{subject}/func"
    data = []
    for irun in range(1, 6):
        data_ = []
        for hemi in ['L', 'R']:
            print(f"  run{irun:02d}, hemi-{hemi}")
            fn = f"{subject}_task-movie_run-{irun:02d}_space-fsaverage6_hemi-{hemi}.func.gii"
            data_.append(load_gifti(os.path.join(datadir, fn)))
        data.append(np.hstack(data_))
    return data


def load_confounds(subject):
    datadir = f"{data_dir}/{subject}/func"
    confounds = []
    confounds_fn = sorted(glob(f'../outputs/fmriprep/{subject}/func/*tsv'))
    for conf in confounds_fn:
        print(conf.split('/')[-1])
        confounds.append(pd.read_csv(conf, sep='\t'))
    return confounds


def clean_data(data, confounds):
    # make predictor matrix using confounds computed by fmriprep
    columns = [
        'framewise_displacement'
    ]
    # compcor
    n_comp_cor = 10
    columns += [f"a_comp_cor_{c:02d}" for c in range(n_comp_cor)]
    # high-pass filtering
    columns += [col for col in confounds.columns if 'cosine' in col]
    
    X = confounds[columns].values
    # remove nans
    X[np.isnan(X)] = 0.
    
    # time to clean up
    # center the data first and store the mean
    data_mean = data.mean(0)
    data = data - data_mean
    coef, _, _, _ = la.lstsq(X, data)
    # remove trends and add back mean of the data
    data_clean = data - X.dot(coef) + data_mean
    return data_clean


def zscore(array):
    array -= array.mean(0)
    array /= (array.std(0) + 1e-8)
    return array

In [5]:
subjects = get_subjects()

In [6]:
print(len(subjects))

25


In [7]:
%%time
data = []
for subject in subjects:
    data_subject = load_data(subject)
    confounds = load_confounds(subject)
    data_subject = np.vstack(
        [zscore(clean_data(dt, conf)) 
         for dt, conf in zip(data_subject, confounds)]
    )
    data.append(data_subject)

Loading data for subject sub-sid000005
  run01, hemi-L
  run01, hemi-R
  run02, hemi-L
  run02, hemi-R
  run03, hemi-L
  run03, hemi-R
  run04, hemi-L
  run04, hemi-R
  run05, hemi-L
  run05, hemi-R
sub-sid000005_task-movie_run-01_desc-confounds_regressors.tsv
sub-sid000005_task-movie_run-02_desc-confounds_regressors.tsv
sub-sid000005_task-movie_run-03_desc-confounds_regressors.tsv
sub-sid000005_task-movie_run-04_desc-confounds_regressors.tsv
sub-sid000005_task-movie_run-05_desc-confounds_regressors.tsv
Loading data for subject sub-sid000007
  run01, hemi-L
  run01, hemi-R
  run02, hemi-L
  run02, hemi-R
  run03, hemi-L
  run03, hemi-R
  run04, hemi-L
  run04, hemi-R
  run05, hemi-L
  run05, hemi-R
sub-sid000007_task-movie_run-01_desc-confounds_regressors.tsv
sub-sid000007_task-movie_run-02_desc-confounds_regressors.tsv
sub-sid000007_task-movie_run-03_desc-confounds_regressors.tsv
sub-sid000007_task-movie_run-04_desc-confounds_regressors.tsv
sub-sid000007_task-movie_run-05_desc-confoun

  run01, hemi-R
  run02, hemi-L
  run02, hemi-R
  run03, hemi-L
  run03, hemi-R
  run04, hemi-L
  run04, hemi-R
  run05, hemi-L
  run05, hemi-R
sub-sid000120_task-movie_run-01_desc-confounds_regressors.tsv
sub-sid000120_task-movie_run-02_desc-confounds_regressors.tsv
sub-sid000120_task-movie_run-03_desc-confounds_regressors.tsv
sub-sid000120_task-movie_run-04_desc-confounds_regressors.tsv
sub-sid000120_task-movie_run-05_desc-confounds_regressors.tsv
Loading data for subject sub-sid000134
  run01, hemi-L
  run01, hemi-R
  run02, hemi-L
  run02, hemi-R
  run03, hemi-L
  run03, hemi-R
  run04, hemi-L
  run04, hemi-R
  run05, hemi-L
  run05, hemi-R
sub-sid000134_task-movie_run-01_desc-confounds_regressors.tsv
sub-sid000134_task-movie_run-02_desc-confounds_regressors.tsv
sub-sid000134_task-movie_run-03_desc-confounds_regressors.tsv
sub-sid000134_task-movie_run-04_desc-confounds_regressors.tsv
sub-sid000134_task-movie_run-05_desc-confounds_regressors.tsv
Loading data for subject sub-sid00014

In [8]:
len(data)

25

In [9]:
print([d.shape for d in data])

[(3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924), (3052, 81924)]


In [14]:
def fast_mean(list_of_arrays):
    mean = np.zeros_like(list_of_arrays[0])
    for array in list_of_arrays:
        mean += array
    mean /= len(list_of_arrays)
    return mean

In [38]:
correlations = []
n_subjects = len(subjects)
n_samples = data[0].shape[0]

for isubject, subject in enumerate(subjects):
    mask_group = np.ones(n_subjects, dtype=bool)
    mask_group[isubject] = False
    mask_group = np.where(mask_group)[0]
    print(f"{subject}: {isubject}, group: {mask_group}")
    data_subject = data[isubject].copy()
    data_group = fast_mean([data[i] for i in mask_group])
    # compute columnwise correlation
    corr = (zscore(data_subject) * zscore(data_group)).sum(0) / n_samples
    correlations.append(corr)

sub-sid000005: 0, group: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
sub-sid000007: 1, group: [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
sub-sid000009: 2, group: [ 0  1  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
sub-sid000010: 3, group: [ 0  1  2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
sub-sid000013: 4, group: [ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
sub-sid000020: 5, group: [ 0  1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
sub-sid000021: 6, group: [ 0  1  2  3  4  5  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
sub-sid000024: 7, group: [ 0  1  2  3  4  5  6  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
sub-sid000025: 8, group: [ 0  1  2  3  4  5  6  7  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
sub-sid000029: 9, group: [ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
sub-sid000

In [39]:
correlations = np.array(correlations)

In [43]:
DIROUT = '../outputs/datapaper/isc'
os.makedirs(DIROUT, exist_ok=True)
np.save(f'{DIROUT}/isc-correlations-all-subjects.npy', correlations)